In [12]:
import random
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector


In [ ]:

def quantum_coin_flip(num_trials):
    alice_wins = 0  # Initialize the counter for Alice's wins
    bob_wins = 0    # Initialize the counter for Bob's wins
    simulator = AerSimulator()  # Create an instance of the AerSimulator for quantum simulation

    for _ in range(num_trials):  # Loop for the specified number of trials
        # Alice prepares the qubit in |+⟩
        qc = QuantumCircuit(1, 1)  # Create a quantum circuit with 1 qubit and 1 classical bit
        qc.h(0)                 # Apply the Hadamard gate to qubit 0, putting it in the |+⟩ state
        

        # Bob receives the qubit and chooses a measurement basis
        bob_choice = random.choice(["X", "Z"])  # Randomly choose between measuring in the X or Z basis
        measurement_result = None              # Initialize a variable to store the measurement result

        if bob_choice == "X":
            # Measure in X-basis
            qc_measure = QuantumCircuit(1, 1)  # Create a new quantum circuit for the X-basis measurement
            qc_measure.h(0)                 # Apply Hadamard to rotate the X basis to the Z basis
            qc_measure.measure(0, 0)        # Measure qubit 0 and store the result in classical bit 0
            combined_qc = qc.compose(qc_measure)  # Combine Alice's preparation circuit with Bob's measurement circuit
            tcirc = transpile(combined_qc, simulator)  # Transpile the combined circuit for the simulator
            result = simulator.run(tcirc, shots=1).result()  # Run the simulation for 1 shot
            counts = result.get_counts(combined_qc)  # Get the measurement counts
            measured_state = list(counts.keys())[0]  # Get the measured state ('0' or '1')
            if measured_state == '0':  # Corresponds to measuring |+⟩ in the X-basis
                bob_wins += 1         # Bob wins
            else:  # Corresponds to measuring |-⟩ in the X-basis
                alice_wins += 1       # Alice wins
            measurement_result = f"X-basis: {'+' if measured_state == '0' else '-'}"  # Format the measurement result

        elif bob_choice == "Z":
            # Measure in Z-basis
            qc_measure = QuantumCircuit(1, 1)  # Create a new quantum circuit for the Z-basis measurement
            qc_measure.measure(0, 0)        # Measure qubit 0 and store the result in classical bit 0
            combined_qc = qc.compose(qc_measure)  # Combine Alice's preparation circuit with Bob's measurement circuit
            tcirc = transpile(combined_qc, simulator)  # Transpile the combined circuit for the simulator
            result = simulator.run(tcirc, shots=1).result()  # Run the simulation for 1 shot
            counts = result.get_counts(combined_qc)  # Get the measurement counts
            measured_state = list(counts.keys())[0]  # Get the measured state ('0' or '1')
            if measured_state == '0':  # Corresponds to measuring |0⟩ in the Z-basis
                bob_wins += 1         # Bob wins
            else:  # Corresponds to measuring |1⟩ in the Z-basis
                alice_wins += 1       # Alice wins
            measurement_result = f"Z-basis: {measured_state}"  # Format the measurement result

        print(f"Bob chose {bob_choice}-basis, measured {measurement_result}")  # Print the outcome of each trial

    bob_win_rate = bob_wins / num_trials  # Calculate Bob's win rate
    alice_win_rate = alice_wins / num_trials  # Calculate Alice's win rate
    return bob_win_rate, alice_win_rate  # Return the win rates


In [ ]:

# Run the game without the twist
num_trials = 500  # Set the number of trials for the game
bob_win_rate, alice_win_rate = quantum_coin_flip(num_trials)  # Run the quantum coin flip game
print(f"\n--- Quantum Coin Flip (No Twist) ---")  
print(f"Number of trials: {num_trials}")  
print(f"Bob's win rate: {bob_win_rate:.3f}") 
print(f"Alice's win rate: {alice_win_rate:.3f}")  



Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose Z-basis, measured Z-basis: 1
Bob chose Z-basis, measured Z-basis: 1
Bob chose X-basis, measured X-basis: +
Bob chose Z-basis, measured Z-basis: 1
Bob chose Z-basis, measured Z-basis: 1
Bob chose X-basis, measured X-basis: +
Bob chose Z-basis, measured Z-basis: 0
Bob chose Z-basis, measured Z-basis: 0
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose Z-basis, measured Z-basis: 0
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose X-basis, measured X-basis: +
Bob chose Z-basis, measur

- _"Bob chose X-basis, measured X-basis: +"_: In this trial, Bob chose to measure the qubit in the X-basis, and the result of the measurement was "+", meaning the qubit was found to be in the |+⟩ state. Since Bob wins if he measures |+⟩ in the X-basis, Bob wins this round.
- _"Bob chose Z-basis, measured Z-basis: 1"_: In this trial, Bob chose to measure in the Z-basis, and the result was "1". Since Bob wins if he measures '0' in the Z-basis, Alice wins this round.

In [ ]:
# --- The Twist: Bob entangles his measurement choice ---

def quantum_coin_flip_with_entanglement(num_trials):
    alice_wins = 0  # Initialize Alice's win counter
    bob_wins = 0    # Initialize Bob's win counter
    simulator = AerSimulator()  # Create an instance of the AerSimulator

    for _ in range(num_trials):  # Loop for the specified number of trials
        # Create a combined circuit with enough qubits (2) and classical bits (1)
        combined_qc = QuantumCircuit(2, 1)  # 2 qubits (Alice's and Bob's ancilla), 1 classical bit for measurement

        # Alice prepares the qubit in |+⟩ (on qubit 0 of combined_qc)
        combined_qc.h(0)  # Apply Hadamard to qubit 0
        

        # Bob's side: Entangled choice and measurement (using qubits 1 and 0 of combined_qc)
        bob_choice_qubit = combined_qc.qubits[1]  # Bob's ancilla qubit for entangled choice
        measurement_qubit = combined_qc.qubits[0]  # Alice's qubit being measured
        classical_bit = combined_qc.clbits[0]  # Classical bit to store the measurement of Alice's qubit

        # Entangle Bob's choice with an ancilla qubit
        combined_qc.h(bob_choice_qubit)  # Put Bob's ancilla qubit into superposition

        # Conditional measurement based on the entangled choice
        combined_qc.cx(bob_choice_qubit, measurement_qubit)  # Apply CNOT controlled by Bob's ancilla, target on Alice's qubit

        # Measure Alice's qubit (qubit 0)
        combined_qc.measure(measurement_qubit, classical_bit)  # Measure Alice's qubit and store the result

        tcirc = transpile(combined_qc, simulator)  # Transpile the combined circuit
        result = simulator.run(tcirc, shots=1).result()  # Run the simulation for 1 shot
        counts = result.get_counts(combined_qc)  # Get the measurement counts
        measurement = list(counts.keys())[0]  # Get the measurement outcome ('0' or '1')

        # Determine Bob's effective measurement basis and the outcome
        if measurement == '0':  # Measurement outcome '0'
            bob_wins += 1     # Bob wins
        else:  # Measurement outcome '1'
            alice_wins += 1   # Alice wins

        print(f"Bob's entangled choice led to measurement: {measurement}")  # Print the measurement outcome

    bob_win_rate = bob_wins / num_trials  # Calculate Bob's win rate
    alice_win_rate = alice_wins / num_trials  # Calculate Alice's win rate
    return bob_win_rate, alice_win_rate

In [ ]:


# Run the game with the twist
num_trials = 500
bob_win_rate_twisted, alice_win_rate_twisted = quantum_coin_flip_with_entanglement(num_trials)
print(f"\n--- Quantum Coin Flip (With Entanglement Twist) ---")
print(f"Number of trials: {num_trials}")
print(f"Bob's win rate: {bob_win_rate_twisted:.3f}")
print(f"Alice's win rate: {alice_win_rate_twisted:.3f}")

Bob's entangled choice led to measurement: 1
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 1
Bob's entangled choice led to measurement: 1
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 1
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 1
Bob's entangled choice led to measurement: 1
Bob's entangled choice led to measurement: 1
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 1
Bob's entangled choice led to measurement: 1
Bob's entangled choice led to measurement: 0
Bob's entangled choice led to measurement: 1
Bob's enta

- _"Bob's entangled choice led to measurement: 1"_: In this trial, after Bob entangled his measurement choice with an ancilla qubit and performed the conditional operation, the measurement of Alice's qubit resulted in the state '1'.
- _"Bob's entangled choice led to measurement: 0"_: In this trial, the final measurement of Alice's qubit resulted in the state '0'.